In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df4 = pd.read_csv('../data/Raw/Study_summary_species.csv')
df4.shape

(3924, 76)

In [3]:
print("LISTA COMPLETA DE VARIABLES")
for i, columna in enumerate(df4.columns):
    print(f"{i+1:2d}. {columna}")

LISTA COMPLETA DE VARIABLES
 1. DOI
 2. Authors
 3. Year
 4. Species
 5. Organism Group
 6. Environment
 7. Life Stage
 8. In vitro/in vivo
 9. Sex
10. Estimated Body Length (cm)
11. Estimated Maximum Ingestible Size (mm)
12. Experiment Type
13. Exposure Route
14. Particle Mix?
15. Negative Control
16. Reference Particle
17. Exposure Media
18. Solvent
19. Detergent
20. pH
21. Salinity (ppt)
22. Temperature (Avg)
23. Temperature (Min)
24. Temperature (Max)
25. Exposure Duration (days)
26. Recovery (Days)
27. Acute/Chronic
28. Number of Doses
29. Replicates
30. Sample Size
31. Dosing Frequency
32. Chemicals Added
33. Added Chemical Dose (nominal)
34. Added Chemical Dose (measured)
35. Plotted Dose Values
36. Unaligned Dose Values
37. Dose Metric
38. Alignment
39. Effect
40. Direction
41. Broad Endpoint Category
42. Specific Endpoint Category
43. Endpoint
44. Level of Biological Organization
45. Target Cell or Tissue
46. Effect Metric
47. Polymer
48. Shape
49. Density (g/cm^3)
50. Density

In [4]:
# Columnas duplicadas
duplicadas = df4.columns[df4.T.duplicated()]
duplicadas

Index(['Unaligned Dose Values'], dtype='object')

In [5]:
[col for col in df4.columns if col.startswith("Unaligned Dose Values")]


['Unaligned Dose Values']

In [6]:
target = "Unaligned Dose Values"

iguales = []

for col in df4.columns:
    if col != target:
        # compara el contenido completo columna a columna
        if df4[col].equals(df4[target]):
            iguales.append(col)

iguales


['Plotted Dose Values']

In [7]:
# Eliminar la columna duplicada en contenido
df4 = df4.drop(columns=["Unaligned Dose Values"])

print("✔ Eliminada columna duplicada: Unaligned Dose Values")


✔ Eliminada columna duplicada: Unaligned Dose Values


In [8]:
df4.duplicated().sum()

np.int64(191)

In [9]:
df4 = df4.drop_duplicates().reset_index(drop=True)
df4.shape

(3733, 75)

### Tratamiento de nulos

In [10]:
def inspect_unique_values(df, max_unique=30):

    print("\n============================")
    print(" INSPECCIÓN DE COLUMNAS POR TIPO ")
    print("============================\n")

    for col in df.columns:
        uniques = df[col].dropna().unique()
        num_unique = len(uniques)

        # ¿Es object?
        if df[col].dtype == "object":
            print(f"\n🔵 OBJECT — {col} ({num_unique} categorías)")
            print(sorted(uniques[:max_unique]))
        
        # ¿Es boolean?
        elif df[col].dtype == "bool":
            print(f"\n🟢 BOOLEAN — {col} ({num_unique} valores)")
            print(sorted(uniques))

        # ¿Es numérica con pocas categorías?
        elif pd.api.types.is_numeric_dtype(df[col]) and num_unique < 20:
            print(f"\n🟣 NUMÉRICA-CATEGÓRICA — {col} ({num_unique} valores)")
            print(sorted(uniques))

        # Si no es categórica, no mostramos nada
        else:
            continue

        print("----------------------------------------")


inspect_unique_values(df4)



 INSPECCIÓN DE COLUMNAS POR TIPO 


🔵 OBJECT — DOI (145 categorías)
['10.1002/etc.3093', '10.1002/etc.3501', '10.1002/etc.3881', '10.1007/s00128-017-2186-3', '10.1007/s11356-016-7584-2', '10.1016/j.aquatox.2019.105296', '10.1016/j.cbpc.2019.02.005', '10.1016/j.chemosphere.2018.05.172', '10.1016/j.ecoenv.2017.03.029', '10.1016/j.ecss.2015.07.023', '10.1016/j.envpol.2014.10.026', '10.1016/j.envpol.2014.12.021', '10.1016/j.envpol.2017.12.088', '10.1016/j.envpol.2018.04.069', '10.1016/j.envpol.2018.11.038', '10.1016/j.envpol.2019.03.085', '10.1016/j.marpolbul.2018.02.039', '10.1016/j.marpolbul.2018.07.067', '10.1016/j.scitotenv.2018.03.176', '10.1021/acs.est.5b04099', '10.1021/acs.est.6b01441', '10.1021/acs.est.8b00168', '10.1021/es504525u', '10.1021/es800249a', '10.1038/s41598-018-33683-6', '10.1038/s41598-019-47160-1', '10.1038/srep41323', '10.1093/mollus/eyx005', '10.1371/journal.pone.0193308', '10.7717/peerj.4601']
----------------------------------------

🔵 OBJECT — Authors (127 cate

In [11]:
#nulos
with pd.option_context('display.max_rows', None):
    display(df4.isna().mean().sort_values(ascending=False) * 100)

Added Chemical Dose (measured)            100.000000
Recovery (Days)                            99.035628
Functional Group                           97.856952
Zeta Potential Media                       92.418966
Charge                                     92.177873
Zeta Potential (mV)                        91.963568
Effect Metric                              82.239486
Temperature (Max)                          80.819716
Temperature (Min)                          80.712564
Salinity (ppt)                             70.345567
pH                                         64.478971
Tissue Distribution                        35.521029
Uptake Validation Method                   32.118939
Estimated Body Length (cm)                 26.064827
Estimated Maximum Ingestible Size (mm)     26.064827
Sample Size                                 5.170104
Density, reported or estimated              2.276989
Dosing Frequency                            1.500134
Uptake Validated?                           1.

In [12]:
cols_to_drop = [
    'Authors',
    'Temperature (Min)',
    'Temperature (Max)',
    'Added Chemical Dose (nominal)',
    'Added Chemical Dose (measured)',
    'Direction'
]

existing = [c for c in cols_to_drop if c in df4.columns]

print("Columnas que serán eliminadas:", existing)

df4 = df4.drop(columns=existing)

print("✔ Limpieza inicial completada")


Columnas que serán eliminadas: ['Authors', 'Temperature (Min)', 'Temperature (Max)', 'Added Chemical Dose (nominal)', 'Added Chemical Dose (measured)', 'Direction']
✔ Limpieza inicial completada


In [13]:
# ================================================
#               CLEANING PIPELINE — df4 
#        Matching the df3 (human) methodology
# ================================================
import re

# --------------------------
# 0. ELIMINACIÓN INICIAL
# --------------------------
cols_to_drop = [
    'Authors',
    'Temperature (Min)',
    'Temperature (Max)',
    'Added Chemical Dose (nominal)',
    'Added Chemical Dose (measured)',
    'Direction'
]

df4 = df4.drop(columns=[c for c in cols_to_drop if c in df4.columns])


# --------------------------
# 1. NORMALIZACIÓN BINARIA
# --------------------------
binary_cols = [
    'Size Validated?',
    'Polymer Validated?',
    'Shape Validated',
    'Sodium Azide Present?',
    'Screened for Chemical Contamination?',
    'Particle Cleaning?',
    'Solvent Rinse',
    'Background Contamination Monitored?',
    'Concentration Validated?',
    'Uptake Validated?',
    'Organisms Fed?'
]

binary_map = {
    'y': 'yes', 'yes': 'yes', 'true': 'yes', '1': 'yes',
    'n': 'no', 'no': 'no', 'false': 'no', '0': 'no',
    'unknown': 'unknown', 'not_reported': 'unknown', 'none': 'unknown'
}

for col in binary_cols:
    if col in df4.columns:
        df4[col] = df4[col].astype(str).str.strip().str.lower()
        df4[col] = df4[col].replace(binary_map)
        df4[col] = df4[col].where(df4[col].isin(['yes','no','unknown']), 'unknown')
        df4[col] = df4[col].replace("nan", np.nan).fillna("unknown")


# --------------------------
# 2. COLUMNAS SENSIBLES
# --------------------------
sensitive_cols = [
    "Functional Group",
    "Target Organelle, Cell, or Tissue",
    "Zeta Potential Media"
]

na_tokens = ["not_reported", "nr", "nan", ".", "-", ""]

for col in sensitive_cols:
    if col in df4.columns:
        df4[col] = df4[col].replace(na_tokens, np.nan)


# --------------------------
# 3. ZETA POTENTIAL (mV)
# --------------------------
col1 = 'Zeta Potential (mV)'
group_col1 = 'Polymer'

if col1 in df4.columns:
    df4[col1] = pd.to_numeric(df4[col1], errors='coerce')
    df4[group_col1] = df4[group_col1].astype(str).str.strip()

    df4['zeta_missing'] = df4[col1].isna().astype(int)
    medians_by_poly = df4.groupby(group_col1)[col1].median()
    df4['zeta_median_by_poly'] = df4[group_col1].map(medians_by_poly)

    global_median = df4[col1].median()
    df4[col1] = df4[col1].fillna(df4['zeta_median_by_poly'])
    df4[col1] = df4[col1].fillna(global_median)

    df4['zeta_imputed_by_group'] = ((df4['zeta_missing'] == 1) &
                                    df4['zeta_median_by_poly'].notna()).astype(int)

    df4['zeta_imputed_by_global'] = ((df4['zeta_missing'] == 1) &
                                     df4['zeta_median_by_poly'].isna()).astype(int)

    df4 = df4.drop(columns=['zeta_median_by_poly'])


# --------------------------
# 4. CHARGE
# --------------------------
df4["charge_missing"] = df4["Charge"].isna().astype(int)
df4["Charge"] = df4["Charge"].fillna("unknown")


# --------------------------
# 5. TEMPERATURE (Avg)
#   same rules as human dataset
# --------------------------
col = "Temperature (Avg)"
mode_col = "In vitro/in vivo"

df4[col] = pd.to_numeric(df4[col], errors="coerce")
df4["in_vitro_flag"] = df4[mode_col].astype(str).str.lower().str.contains("in vitro")

# In Vitro → 37°C
df4.loc[df4["in_vitro_flag"] & df4[col].isna(), col] = 37.0

# In Vivo → mediana por species
species_medians = df4.groupby("Species")[col].median()
mask = df4[col].isna()
df4.loc[mask, col] = df4.loc[mask, "Species"].map(species_medians)

# Fallback global
df4[col] = df4[col].fillna(df4[col].median())


# --------------------------
# 6. SEX
# --------------------------
df4["Sex"] = df4["Sex"].replace({"Not Reported": np.nan})

# In Vitro → not_applicable
df4.loc[df4["in_vitro_flag"] & df4["Sex"].isna(), "Sex"] = "not_applicable"

# In Vivo → unknown
df4.loc[~df4["in_vitro_flag"] & df4["Sex"].isna(), "Sex"] = "unknown"


# --------------------------
# 7. PLOTTED DOSE VALUES (equivalente a Selected Dose)
# --------------------------
col = "Plotted Dose Values"

df4[col] = pd.to_numeric(df4[col], errors='coerce')

df4[col] = df4.groupby("DOI")[col].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
)


# --------------------------
# 8. PARTICLE MASS (mg)
# --------------------------
col = "Particle Mass (mg)"
group = "Polymer"

df4[col] = pd.to_numeric(df4[col], errors="coerce")
df4["particle_mass_missing"] = df4[col].isna().astype(int)

med_poly = df4.groupby(group)[col].median()
df4["mass_median_by_poly"] = df4[group].map(med_poly)
global_median = df4[col].median()

df4[col] = df4[col].fillna(df4["mass_median_by_poly"])
df4[col] = df4[col].fillna(global_median)

df4["particle_mass_imputed_by_group"] = (
    (df4["particle_mass_missing"] == 1) &
    df4["mass_median_by_poly"].notna()
).astype(int)

df4["particle_mass_imputed_by_global"] = (
    (df4["particle_mass_missing"] == 1) &
    df4["mass_median_by_poly"].isna()
).astype(int)

df4 = df4.drop(columns=["mass_median_by_poly"])


# --------------------------
# 9. PARTICLE VOLUME (μm³)
# --------------------------
vol_col = "Particle Volume (μm^3)"
size_col = "Size Category"

df4[size_col] = df4[size_col].astype(str).str.strip().replace({"nan": np.nan})
df4[vol_col] = pd.to_numeric(df4[vol_col], errors="coerce")

df4["particle_volume_missing"] = df4[vol_col].isna().astype(int)

med_size = df4.groupby(size_col)[vol_col].median()
df4["vol_median_by_size"] = df4[size_col].map(med_size)

global_median = df4[vol_col].median()
df4[vol_col] = df4[vol_col].fillna(df4["vol_median_by_size"])
df4[vol_col] = df4[vol_col].fillna(global_median)

df4 = df4.drop(columns=["vol_median_by_size"])


# --------------------------
# 10. PARTICLE LENGTH
# --------------------------
col = "Particle Length (μm)"
shape = "Shape"

df4[shape] = df4[shape].astype(str).str.strip().replace({"Not Reported": np.nan})
df4['particle_length_missing'] = df4[col].isna().astype(int)

median_global = df4[col].median()
median_by_shape = df4.groupby(shape)[col].median()

df4[col] = df4.apply(
    lambda r: r[col] if pd.notna(r[col])
    else (median_by_shape[r[shape]] if pd.notna(r[shape]) and pd.notna(median_by_shape.get(r[shape])) else median_global),
    axis=1
)


# --------------------------
# 11. EXPOSURE MEDIA
# --------------------------
df4["Exposure Media"] = df4["Exposure Media"].replace("nan", np.nan).fillna("not_reported")


# --------------------------
# 12. REPLICATES — moda global
# --------------------------
col = "Replicates"
df4[col] = pd.to_numeric(df4[col], errors="coerce")
df4[col] = df4[col].fillna(df4[col].mode()[0])


# --------------------------
# 13. DENSITY — mediana global
# --------------------------
col = "Density (g/cm^3)"
df4[col] = pd.to_numeric(df4[col], errors="coerce")
df4[col] = df4[col].fillna(df4[col].median())


# --------------------------
# 14. SAMPLE SIZE — mediana por Experiment Type
# --------------------------
col = "Sample Size"
df4[col] = pd.to_numeric(df4[col], errors="coerce")

df4["sample_size_missing"] = df4[col].isna().astype(int)

df4[col] = df4.groupby("Experiment Type")[col].transform(lambda x: x.fillna(x.median()))
df4[col] = df4[col].fillna(df4[col].median())

if df4[col].dropna().apply(float.is_integer).all():
    df4[col] = df4[col].astype("Int64")


In [14]:
df4 = df4.drop(columns=[
    "Recovery (Days)",
    "Tissue Distribution",
    "Uptake Validation Method", 
    "pH",
    "Salinity (ppt)"
])

In [15]:
# Functional Group — 97.9% NA. imputar not_reported
df4["Functional Group"] = df4["Functional Group"].fillna("not_reported")

In [16]:
# Zeta Potential Media — 92.4% NA, imputar not_reported
df4["Zeta Potential Media"] = df4["Zeta Potential Media"].fillna("not_reported")

In [17]:
# Effect Metric — 82.2% NA, imputar not_reported
df4["Effect Metric"] = df4["Effect Metric"].fillna("not_reported")

In [18]:
# Density, reported or estimated — 2.27% NA, imputar con Normalizar string, Inferir “reported” si tiene valor numérico en Density, Todo lo demás → “unknown”
col_cat = "Density, reported or estimated"
col_num = "Density (g/cm^3)"

df4[col_cat] = df4[col_cat].astype(str).str.strip().str.lower().replace({"nan": np.nan})

df4[col_cat] = df4[col_cat].replace({
    "est": "estimated", "estim": "estimated",
    "rep": "reported",  "rpt": "reported"
})

mask_infer = df4[col_cat].isna() & df4[col_num].notna()
df4.loc[mask_infer, col_cat] = "reported"

df4[col_cat] = df4[col_cat].fillna("unknown")


In [19]:
# Dosing Frequency — 1.50% NA, imputar mediana global
df4["Dosing Frequency"] = pd.to_numeric(df4["Dosing Frequency"], errors="coerce")
df4["Dosing Frequency"] = df4["Dosing Frequency"].fillna(df4["Dosing Frequency"].median())

In [20]:
# Imputación Estimated Maximum Ingestible Size (mm)

col = "Estimated Maximum Ingestible Size (mm)"
group = "Species"

# Convertir a numérico
df4[col] = pd.to_numeric(df4[col], errors="coerce")

# Flag de missing original
df4["estimated_max_ingestible_missing"] = df4[col].isna().astype(int)

# Mediana por Species
median_by_species = df4.groupby(group)[col].median()

# Imputación por especie
mask_missing = df4[col].isna()
df4.loc[mask_missing, col] = df4.loc[mask_missing, group].map(median_by_species)

# Fallback global
global_median = df4[col].median(skipna=True)
df4[col] = df4[col].fillna(global_median)

# Resumen
print("=== Imputación Estimated Maximum Ingestible Size (mm) ===")
print("Missing originales:", df4["estimated_max_ingestible_missing"].sum())
print("Mediana global usada en fallback:", global_median)
print("Missing tras imputación:", df4[col].isna().sum())
print("Valores únicos (primeros 20):", df4[col].unique()[:20])


=== Imputación Estimated Maximum Ingestible Size (mm) ===
Missing originales: 973
Mediana global usada en fallback: 2.3794919391134903
Missing tras imputación: 0
Valores únicos (primeros 20): [0.34112055 0.0615852  0.51677985 8.17874834 0.55997704 6.12159942
 2.37949194 0.77279304 1.81877495 0.24446346 7.66828416 5.07525571
 0.02070093 5.49546286 2.7388472  4.17378317 5.60015439 0.04339831
 0.10387193 0.21167958]


In [21]:
# Imputación Estimated Body Length (cm)

col = "Estimated Body Length (cm)"
group = "Species"

# Convertir a numérico correctamente
df4[col] = pd.to_numeric(df4[col], errors="coerce")

# Flag de missing original
df4["estimated_body_length_missing"] = df4[col].isna().astype(int)

# Calcular mediana por Species
median_by_species = df4.groupby(group)[col].median()

# Imputar por mediana de la especie
mask_missing = df4[col].isna()
df4.loc[mask_missing, col] = df4.loc[mask_missing, group].map(median_by_species)

# Fallback global por si quedan NA
global_median = df4[col].median(skipna=True)
df4[col] = df4[col].fillna(global_median)

# Resumen
print("=== Imputación Estimated Body Length (cm) ===")
print("Missing original:", df4['estimated_body_length_missing'].sum())
print("Mediana global usada como fallback:", global_median)
print("Missing tras imputación:", df4[col].isna().sum())
print("Valores únicos (primeros 20):", df4[col].unique()[:20])


=== Imputación Estimated Body Length (cm) ===
Missing original: 973
Mediana global usada como fallback: 4.0
Missing tras imputación: 0
Valores únicos (primeros 20): [ 0.5     0.08    0.78   15.      0.85   11.      4.      1.2     3.
  0.35   14.      9.      0.0249  9.8     4.65    7.3    10.      0.055
  0.14    0.3   ]


In [22]:
#nulos
with pd.option_context('display.max_rows', None):
    display(df4.isna().mean().sort_values(ascending=False) * 100)

DOI                                       0.0
Year                                      0.0
Species                                   0.0
Organism Group                            0.0
Environment                               0.0
Life Stage                                0.0
In vitro/in vivo                          0.0
Sex                                       0.0
Estimated Body Length (cm)                0.0
Estimated Maximum Ingestible Size (mm)    0.0
Experiment Type                           0.0
Exposure Route                            0.0
Particle Mix?                             0.0
Negative Control                          0.0
Reference Particle                        0.0
Exposure Media                            0.0
Solvent                                   0.0
Detergent                                 0.0
Temperature (Avg)                         0.0
Exposure Duration (days)                  0.0
Acute/Chronic                             0.0
Number of Doses                   

In [23]:
df4['Effect Metric'].unique()

array(['not_reported', 'NOEC', 'LOEC', 'HONEC'], dtype=object)

In [25]:
df4_processed = df4
df4_processed.to_csv("../data/processed/species_summary_processed.csv", index=False)